# Assignment 3_2: Echo State Networks

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import torch
import torch.utils.data as data
import torch.nn.functional as F
import torch.optim as optim

from esn import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
narma_df = pd.read_csv('../NARMA10.csv', header=None)
narma_df.iloc[:, :20] # visualize the first 20 columns

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.083964,0.48934,0.35635,0.25024,0.23554,0.029809,0.34099,0.021216,0.035723,0.26082,0.048365,0.40907,0.40877,0.36122,0.074933,0.32980,0.25930,0.48649,0.32450,0.40017
1,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.132850,0.17536,0.37127,0.36481,0.337070,0.20447,0.33003,0.20726,0.18825,0.28343


In [3]:
x_data = torch.tensor(narma_df.iloc[0].values, dtype=torch.float32) # float 32 for better memory efficiency
y_data = torch.tensor(narma_df.iloc[1].values, dtype=torch.float32)

# 4000 tr, 1000 val, 5000 test (WARNING: load entire dataset in memory ONLY because it is small and also the NN is quite small)
dev_x, dev_y = x_data[:5000], y_data[:5000] # only used for retraining (train + val sets)

test_x, test_y = x_data[5000:], y_data[5000:]

train_x, val_x = dev_x[:4000], dev_x[4000:]
train_y, val_y = dev_y[:4000], dev_y[4000:]

In [5]:
train_x = train_x.unsqueeze(1).unsqueeze(1)
train_x.shape

torch.Size([4000, 1, 1])

In [6]:
val_x = val_x.unsqueeze(1).unsqueeze(1)
val_x.shape

torch.Size([1000, 1, 1])

In [19]:
hyperparams = {
    'hidden_size': [256, 512, 1024], 
    'ridge_regression':  [1e-6],
    'omhega_in': [0.5],
    'omhega_b': [0.3],
    'rho': [0.7, 0.8, 0.9]
}

model_selection_history = grid_search(hyperparams, train_x, train_y, val_x, val_y, n_iter=5) 

df = pd.DataFrame.from_dict(model_selection_history, orient='index')
df.to_csv('esn_grid_search.csv')

,hidden_size,ridge_regression,omhega_in,omhega_b,rho,train_mse_mean,train_mse_var,val_mse_mean,val_mse_var
config_0,256,0.000001,0.5,0.3,0.7,9.521449e-05,2.054162e-09,0.000107,1.819645e-09
config_1,256,0.000001,0.5,0.3,0.8,3.507915e-05,2.863444e-11,0.000043,5.412783e-11
config_2,256,0.000001,0.5,0.3,0.9,4.742078e-05,1.243398e-10,0.000058,1.800040e-10
config_3,512,0.000001,0.5,0.3,0.7,8.129021e-06,2.014190e-12,0.000010,4.469299e-12
config_4,512,0.000001,0.5,0.3,0.8,3.399504e-06,3.999810e-13,0.000005,7.536661e-13
config_5,512,0.000001,0.5,0.3,0.9,6.621438e-06,2.228957e-12,0.000010,2.671556e-12
config_6,1024,0.000001,0.5,0.3,0.7,1.831695e-06,4.374816e-14,0.000003,1.533532e-13
config_7,1024,0.000001,0.5,0.3,0.8,6.905953e-07,9.050000e-15,0.000001,4.099552e-14
config_8,1024,0.000001,0.5,0.3,0.9,7.889038e-07,9.260218e-15,0.000002,4.217145e-14


In [ ]:
df_results = pd.read_csv('esn_grid_search.csv', index_col=0)
df_results.sort_values(by='val_mse_mean', ascending=True).head(10)